# Linear regression
In the will will going to examine some ecommerce customer data for a company's website and mobile app.We are going  build a regression model that will predict the customer's yearly spend on the company's product.

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('regression').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [5]:
# Use Spark to read in the Ecommerce Customers csv file.
data = spark.read.csv('/home/shrinath/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/Ecommerce_Customers.csv',
                    inferSchema= True, header=True)

In [9]:
# Print the Schema of the DataFrame
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [8]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [12]:
data.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

# Setting Up DataFrame for Machine Learning

In [25]:
# A few things we need to do before Spark can accept the data!
# It needs to be in the form of two columns
# ("label","features")

# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [14]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [26]:
# Here we are only focusing on numerical features with that trying to pridict the outcomes
assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App", 
               "Time on Website",'Length of Membership'],
    outputCol="features")

In [28]:
output = assembler.transform(data)

In [32]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [31]:
output.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [37]:
final_data = output.select("features",'Yearly Amount Spent')

In [38]:
train_data,test_data = final_data.randomSplit([0.7,0.3])


In [39]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                343|
|   mean| 494.69897852683096|
| stddev|  80.27382867619237|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+



In [40]:
# Create a Linear Regression Model object
lr=LinearRegression(labelCol='Yearly Amount Spent')

In [42]:
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data,)

In [43]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [26.955494735118243,39.24061275812557,0.9422394610939059,61.30832447051788] Intercept: -1115.943247323403


In [44]:
test_results = lrModel.evaluate(test_data)

In [45]:
# Interesting results....
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|  15.05607095321011|
|  3.457638044395196|
|-14.978835121788393|
| 10.015263036042825|
|0.19078497288944618|
| 24.655341852439506|
|  5.268617223154365|
|  5.708344213749001|
|  6.087190286565715|
| 3.3828947119864665|
| 18.454628972602052|
|-0.5282920843328043|
|   5.48834954670798|
| -3.999936804422134|
|  9.332844813016607|
|  9.783127316304274|
|-3.5952500292884793|
| 13.173921085056065|
|-12.674786561658493|
|  4.334232914650272|
+-------------------+
only showing top 20 rows



In [46]:
unlabeled_data = test_data.select('features')

In [47]:
predictions = lrModel.transform(unlabeled_data)

In [48]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 393.5842801194174|
|[30.5743636841713...|438.60677571367046|
|[30.8162006488763...| 281.0651760702574|
|[30.9716756438877...| 484.6233467208499|
|[31.2606468698795...| 421.1358462840619|
|[31.2834474760581...|  567.125747573228|
|[31.3091926408918...|427.45210061677926|
|[31.3584771924370...| 489.4676062357264|
|[31.3662121671876...| 424.5016922699192|
|[31.5316044825729...| 433.1327110173761|
|[31.6098395733896...| 426.0909206785061|
|[31.7216523605090...|348.30521871620545|
|[31.7366356860502...|491.44509670882394|
|[31.7656188210424...|500.55401844002927|
|[31.8209982016720...|415.34243620019674|
|[31.8512531286083...| 463.2091193504941|
|[31.8745516945853...|  395.880494275556|
|[31.9262720263601...|379.03101235927033|
|[31.9365486184489...| 439.8741714569867|
|[32.0123007682454...|488.61082015130796|
+--------------------+------------

In [49]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 10.869602266521543
MSE: 118.14825343237025
